# RNN #

In [3]:
import torch
import pandas as pd
import os

## 資料預處理 ##

取得資料夾所有yml資料

In [5]:
dir_path = r"D:\ml_data\data_set\chatterbot-corpus-master\chatterbot-corpus-master\chatterbot_corpus\data\chinese"
file_list = os.listdir(dir_path)
list_ = []
for file in file_list:
    if '.yml' in file:
        list_.append(file)
file_list = list_

取得合併資料夾所有資料

In [3]:
file_merge = ""
for file_name in file_list: 
    file_path = os.path.join(dir_path,file_name)
    with open(file_path,'r',encoding='utf-8') as f:
        file_merge += f.read()

In [4]:
print(file_merge[:128])

categories:
- AI
conversations:
- - 什么是ai
  - 人工智能是工程和科学的分支,致力于构建思维的机器。
- - 你写的是什么语言
  - 蟒蛇
- - 你听起来像数据
  - 是的,我受到指挥官数据的人工个性的启发



去除類別，對話整理成"成對"

In [11]:
import re
import numpy as np
data_list = np.array(file_merge.split('\n'))
data_list_new = []
A_ready = False
content = None
for data_line in data_list:
    if "- - " in data_line:
        if A_ready == False:
            content = data_line
        data_list_new.append(data_line)
        A_ready = True        
    if "  - " in data_line:
        if A_ready == False:
            data_list_new.append(content)
            data_list_new.append(data_line)
        if A_ready == True:
            data_list_new.append(data_line)
        A_ready = False
data_list = data_list_new
num = 1126
print(data_list[num])
print(data_list[num+1])

- - 海豚使用的一种感觉，类似于声纳，以确定附近的物品的位置和形状.它是什么
  - 回声定位


列表轉回字串

In [35]:
file_str = ''
for list_str in data_list:
    file_str += list_str+'\n'
print(file_str[:123])

- - 什么是ai
  - 人工智能是工程和科学的分支,致力于构建思维的机器。
- - 你写的是什么语言
  - 蟒蛇
- - 你听起来像数据
  - 是的,我受到指挥官数据的人工个性的启发
- - 你是一个人工语言实体
  - 那是我的名字。



簡轉繁

In [46]:
from opencc import OpenCC
cc = OpenCC('s2t')
file_str =  cc.convert(file_str)
print(file_str[:123])

- - 什麼是ai
  - 人工智能是工程和科學的分支,致力於構建思維的機器。
- - 你寫的是什麼語言
  - 蟒蛇
- - 你聽起來像數據
  - 是的,我受到指揮官數據的人工個性的啓發
- - 你是一個人工語言實體
  - 那是我的名字。



存檔

In [41]:
file_path = os.path.join(dir_path,"all_categories.txt")
with open(file_path,'w',encoding='utf-8') as f:
    f.write(file_str)

jieba分詞

In [78]:
import jieba
import logging
import time

def main(file_path,save_path):
    #設置log格式，以及print的log等級
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # jieba custom setting.
    jieba.set_dictionary('jieba_dict/dict.txt.big')

    # load stopwords set
    #將停用詞每row分別加進集合
    stopword_set = set()
    with open('jieba_dict/stop_word1_zh.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip('\n'))   #移除頭尾換行 strip('\n')
    output = open(save_path, 'w', encoding='utf-8')
    with open(file_path, 'r', encoding='utf-8') as content :
        #每一行都切成一個iter
        for texts_num, line in enumerate(content):
            line = line.strip('\n')
            words = jieba.cut(line, cut_all=False,HMM=True)    #進行斷詞
            for word in words:
                #依每個詞判斷是否為停用詞(不是就寫入)
                if word not in stopword_set:
                    output.write(word + ' ')     #每一行的iter(詞)以空格隔開
            output.write('\n')      #iter完以換行符區隔

            if (texts_num + 1) % 10000 == 0:
                logging.info("已完成前 %d 行的斷詞" % (texts_num + 1))
    output.close()

if __name__ == '__main__':
    file_path = r'D:\ml_data\data_set\chatterbot-corpus-master\chatterbot-corpus-master\chatterbot_corpus\data\chinese\all_categories.txt'
    save_path = r'D:\ml_data\data_set\chatterbot-corpus-master\chatterbot-corpus-master\chatterbot_corpus\data\chinese\all_categories_jieba.txt'
    main(file_path,save_path)

Building prefix dict from C:\Users\Jhin\Jupyter_notebook\save\jieba_dict\dict.txt.big ...
2019-02-27 18:49:15,423 : DEBUG : Building prefix dict from C:\Users\Jhin\Jupyter_notebook\save\jieba_dict\dict.txt.big ...
Loading model from cache C:\Users\Jhin\AppData\Local\Temp\jieba.u18faab2d92f2477ea673e5e6e60014f3.cache
2019-02-27 18:49:15,425 : DEBUG : Loading model from cache C:\Users\Jhin\AppData\Local\Temp\jieba.u18faab2d92f2477ea673e5e6e60014f3.cache
Loading model cost 1.023 seconds.
2019-02-27 18:49:16,449 : DEBUG : Loading model cost 1.023 seconds.
Prefix dict has been built succesfully.
2019-02-27 18:49:16,451 : DEBUG : Prefix dict has been built succesfully.


讀取檔案

In [140]:
file_path = os.path.join(dir_path,"all_categories_jieba.txt")
with open(file_path,'r',encoding='utf-8') as f:
    data = f.read()

In [141]:
data_list = data.split('\n')[:1128]

In [146]:
data = np.array(data_list).reshape(-1,2)

array([['- - 什麼 是 ai ', '- 人工智能 是 工程 和 科學 的 分支 致力於 構建 思維 的 機器 '],
       ['- - 你 寫 的 是 什麼 語言 ', '- 蟒蛇 '],
       ['- - 你 聽 起來 像 數據 ', '- 是 的 我 受到 指揮官 數據 的 人工 個性 的 啓發 '],
       ...,
       ['- - 天佑 女王 是 哪個 國家 的 國歌 ', '- 大不列顛 聯合王國 '],
       ['- - 凱爾特 陸棚 是 什麼 大陸 的 大陸架 的 一部分 ', '- 歐洲 '],
       ['- - 海豚 使用 的 一種 感覺 類似 於 聲納 以 確定 附近 的 物品 的 位置 和 形狀 它 是 什麼 ',
        '- 回聲 定位 ']], dtype='<U355')

In [170]:
x = data[:,[0]].reshape(-1)
y = data[:,[1]].reshape(-1)

In [171]:
x_ = []
y_ = []
for string in x:
    x_.append(string[4:])
x = np.array(x_)
for string in y:
    y_.append(string[2:])
y = np.array(y_)

In [175]:
for num in range(5,10):
    print(x[num],'\n',y[num])

你 沒有 意義  
 恰恰相反 這 一切 都 對 我 的 假想 心理 有 意義 
你 沒有 意義  
 這 一切 對 我 的 人造 大腦 都 有 意義 
你 是 不朽 的  
 不是 但 我 可以 永遠 永遠 
你 沒有 任何 意義  
 這 一切 都 對 我 的 人造 心靈 有 意義 
你 不能 克隆  
 軟件 複製 是 數字 克隆 的 一種 形式 


In [ ]:
#簡繁轉換

In [1]:
import torch
import pandas as pd
import os
from opencc import OpenCC
import re

In [2]:
def get_dir_file_list(dir_path,filter=None):
    """
    filter: if filter in file: list_.append(file)
    """
    dir_path = dir_path
    file_list = os.listdir(dir_path)
    list_ = []
    if filter == None:
        for file in file_list:
            list_.append(file)
    else:
        for file in file_list:
            if filter in file:
                list_.append(file)
    file_list = list_
    return file_list

In [55]:
dir_path = r"D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\chat"
file_list = get_dir_file_list(dir_path)
file_list

['all_categories.txt', 'hit_on01.txt', 'xiaohuangji50w_nofenci_zh_jieba.conv']

In [56]:
for f_name in file_list[:]:
    cc = OpenCC('s2t')
    file_path = os.path.join(dir_path,f_name)
    save_path = os.path.join(dir_path,'new_'+f_name)
    with open(file_path,'r',encoding='utf-8') as read_f:
        with open(save_path,'w',encoding='utf-8') as write_f:
            for read_line in read_f:
                file_str =  cc.convert(read_line)
                write_f.writelines(file_str)

In [ ]:
#去除utf-8 BOM

In [174]:
with open(r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\all_categories.txt',
          'rb') as read_f:
    read_f.read(3)
    with open(r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\all_categories1.txt',
              'wb') as write_f:
        str_ = read_f.read(4096)
        while str_ != b'':
            write_f.write(str_)
            str_ = read_f.read(4096)

In [176]:
list1 = []
with open(r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\all_categories1.txt',
          'r',encoding='utf-8') as read_f:
    for file_str in read_f:
        list1.append(file_str.strip())

In [177]:
list1[-10:]

['哈勃太空望遠鏡，於1990年發射進入近地軌道，它是以什麼美國天文學家命名的?',
 '愛德溫·哈伯',
 '離銀河系最近的大型星系叫什麼?',
 '仙女座星系.',
 '天佑女王是哪個國家的國歌?',
 '大不列顛聯合王國',
 '凱爾特陸棚，是什麼大陸的大陸架的一部分?',
 '歐洲',
 '海豚使用的一種感覺，類似於聲納，以確定附近的物品的位置和形狀.它是什麼',
 '回聲定位']

In [ ]:
#批量分詞

In [15]:
import torch
import pandas as pd
import os
from opencc import OpenCC
import re
import logging
import jieba

In [57]:
def get_dir_file_list(dir_path,filter=None):
    """
    filter: if filter in file: list_.append(file)
    """
    dir_path = dir_path
    file_list = os.listdir(dir_path)
    list_ = []
    if filter == None:
        for file in file_list:
            list_.append(file)
    else:
        for file in file_list:
            if filter in file:
                list_.append(file)
    file_list = list_
    return file_list

def main(file_path,save_path):
    #設置log格式，以及print的log等級
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # jieba custom setting.
    jieba.set_dictionary('jieba_dict/dict.txt.big')

    # load stopwords set
    #將停用詞每row分別加進集合
    stopword_set = set()
    #設置停用詞讀取路徑
    with open('jieba_dict/robot_stop_1.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip('\n'))   #移除頭尾換行 strip('\n')
    output = open(save_path, 'w', encoding='utf-8')
    with open(file_path, 'r', encoding='utf-8') as content :
        #每一行都切成一個iter
        for texts_num, line in enumerate(content):
            line = line.strip('\n')
            words = jieba.cut(line, cut_all=False,HMM=True)    #進行斷詞
            for word in words:
                #依每個詞判斷是否為停用詞(不是就寫入)
                if word not in stopword_set:
                    output.write(word + ' ')     #每一行的iter(詞)以空格隔開
            output.write('\n')      #iter完以換行符區隔

            if (texts_num + 1) % 10000 == 0:
                logging.info("已完成前 %d 行的斷詞" % (texts_num + 1))
    output.close()

if __name__ == '__main__':
    chat_sentence_dir = r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\chat'
    chat_sentence_file_name_list = get_dir_file_list(chat_sentence_dir)
    for file_name in chat_sentence_file_name_list:
        chat_sentence_path = os.path.join(chat_sentence_dir,file_name)
        chat_sentence_jieba_path = os.path.join(chat_sentence_dir, 'jieba_'+file_name)
        save_path = chat_sentence_jieba_path
        main(chat_sentence_path,save_path)

Building prefix dict from C:\Users\Jhin\Jupyter_notebook\save\jieba_dict\dict.txt.big ...
2019-03-10 00:36:24,445 : DEBUG : Building prefix dict from C:\Users\Jhin\Jupyter_notebook\save\jieba_dict\dict.txt.big ...
Loading model from cache C:\Users\Jhin\AppData\Local\Temp\jieba.u18faab2d92f2477ea673e5e6e60014f3.cache
2019-03-10 00:36:24,447 : DEBUG : Loading model from cache C:\Users\Jhin\AppData\Local\Temp\jieba.u18faab2d92f2477ea673e5e6e60014f3.cache
Loading model cost 1.020 seconds.
2019-03-10 00:36:25,467 : DEBUG : Loading model cost 1.020 seconds.
Prefix dict has been built succesfully.
2019-03-10 00:36:25,468 : DEBUG : Prefix dict has been built succesfully.
Building prefix dict from C:\Users\Jhin\Jupyter_notebook\save\jieba_dict\dict.txt.big ...
2019-03-10 00:36:25,534 : DEBUG : Building prefix dict from C:\Users\Jhin\Jupyter_notebook\save\jieba_dict\dict.txt.big ...
Loading model from cache C:\Users\Jhin\AppData\Local\Temp\jieba.u18faab2d92f2477ea673e5e6e60014f3.cache
2019-03-10

In [ ]:
#train

In [4]:
import logging

from gensim.models import word2vec

def main(file_path,save_path):

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    # https://radimrehurek.com/gensim/models/word2vec.html
    #單檔案
#     sentences = word2vec.LineSentence(file_path)
    #多檔案
    sentences = word2vec.PathLineSentences(file_path)
    model = word2vec.Word2Vec(sentences, size=300,window=10,min_count=5)

    #保存模型，供日後使用
    model.save(save_path)

    #模型讀取方式
    # model = word2vec.Word2Vec.load("your_model_name")

if __name__ == "__main__":
    file_path = r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence'
    save_path = r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence\wiki_word2vec0.model'
    main(file_path,save_path)

In [ ]:
#讀取測試

In [1]:
from gensim.models import word2vec
if __name__ == "__main__":
    save_path = r'D:\Backup\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence\wiki_word2vec0.model'
    model = word2vec.Word2Vec.load(save_path)

In [2]:
model.wv.most_similar(positive=[',','。'],negative=['.'],topn=5)
model.wv.most_similar(positive=['北京','日本'],negative=['東京'],topn=5)

[('中國', 0.6789283156394958),
 ('中華人民共和國', 0.5194883346557617),
 ('天津', 0.5084283351898193),
 ('上海', 0.49704641103744507),
 ('中國政府', 0.49614858627319336)]

In [3]:
model.wv.most_similar(positive=',',topn=5)

[('tons', 0.6170707941055298),
 ('萬零', 0.5534135103225708),
 ('0k', 0.5509641170501709),
 ('架次', 0.5040297508239746),
 ('￥', 0.5037356019020081)]

In [4]:
model.wv.index2word[0:5]

['，', '的', '。', '、', '年']

In [5]:
vec = model.wv.get_vector(',')
print(vec)

[ 1.7720475e+00 -3.1741173e+00  2.9003337e-01 -2.0293701e+00
 -1.8457556e+00 -1.1188092e+00  1.4739352e-01  4.1038127e+00
 -1.0837914e+00  9.3161717e-02  1.7648523e+00  2.9613364e+00
 -1.6822392e-01 -2.2486007e+00  9.9281293e-01 -1.5263356e+00
  2.0999648e-01 -1.3027124e+00  1.6743528e+00 -9.3285924e-01
 -1.6102500e+00  2.6512172e+00  2.2831595e+00  1.5537627e+00
  2.5151241e+00  1.8922043e+00 -1.9985626e+00 -8.5118723e-01
  2.4825826e+00 -1.8364611e-01 -1.4432459e+00 -1.0923058e+00
  8.2405984e-01  3.7991204e+00  7.1566987e-01  1.6480860e+00
  4.4686875e+00 -2.9307039e+00 -2.0631073e-01 -1.5097848e-01
  3.7871653e-01  2.1187770e+00 -2.5953493e-01 -1.3296324e+00
 -1.0261503e+00 -1.8489912e-01 -1.4288993e+00 -1.1781234e+00
  2.0210264e+00  5.5016816e-01 -5.9882963e-01 -1.1037207e+00
 -2.1071701e+00 -2.5863230e+00  1.7657706e+00  4.9389446e-01
 -5.9311020e-01  6.9572248e-02 -1.0718168e+00  9.3465364e-01
 -5.8814511e+00 -1.2960665e+00 -5.0631350e-01 -3.9529088e-01
 -5.7256670e+00  2.24313

In [6]:
model.wv.similar_by_vector(vec)

[(',', 1.0),
 ('tons', 0.6170707941055298),
 ('萬零', 0.5534135103225708),
 ('0k', 0.5509641170501709),
 ('架次', 0.5040297508239746),
 ('￥', 0.5037356019020081),
 ('輛', 0.494095116853714),
 ('013', 0.4934525787830353),
 ('AWG', 0.4926324188709259),
 ('+', 0.49228981137275696)]

In [8]:
import numpy as np
zero_vec = np.zeros(300)
print(model.wv.similar_by_vector(zero_vec,topn=5))

[('婚史', 0.0), ('triangular', 0.0), ('邁杉', 0.0), ('甘尼許', 0.0), ('齊威', 0.0)]


In [ ]:
#讀取後 加新詞

In [1]:
from gensim.models import word2vec
import logging
import sys
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
save_path = r'D:\Backup\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence\wiki_word2vec0.model'
model = word2vec.Word2Vec.load(save_path)

In [2]:
print(len(model.wv.index2word))
try:
    model.wv.most_similar(positive='woof',topn=5)
except KeyError as err:
    print(err)
try:
    model.wv.most_similar(positive='傻眼貓咪',topn=5)
except KeyError as err:
    print(err)

2019-03-19 16:13:41,511 : INFO : precomputing L2-norms of word weight vectors


692776
"word 'woof' not in vocabulary"
"word '傻眼貓咪' not in vocabulary"


In [3]:
#版本1
model.vocabulary.min_count = 1
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model.build_vocab(sentences, update=True)#训练该行
model.train(sentences,total_examples= model.corpus_count,epochs= model.epochs)
print(len(model.wv.index2word))
model.wv.most_similar(positive='woof',topn=5)

692778


[('糾紛案件', 0.2756342887878418),
 ('根本原因', 0.27125775814056396),
 ('離散性', 0.2641603648662567),
 ('金融市場', 0.2583328187465668),
 ('隨機變量', 0.25302454829216003)]

In [3]:
#版本2
model.vocabulary.min_count = 1   #修改之前訓練時的設定(新增的句子較少所以 詞的最小出現次數改為1)
file_path = r'D:\Backup\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence\test.txt'
sentences = word2vec.LineSentence(file_path)
model.build_vocab(sentences, update=True)#训练该行
newmodel = model.train(sentences,total_examples= model.corpus_count,epochs= model.epochs)
print(len(model.wv.index2word))
model.wv.most_similar(positive='傻眼貓咪',topn=5)

692778


[('Zeonic', 0.2712220251560211),
 ('自衛權', 0.25267353653907776),
 ('通比', 0.23276172578334808),
 ('第一次', 0.2252526432275772),
 ('利載', 0.21481187641620636)]

In [2]:
#版本3
file_path = r'D:\Backup\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence\test.txt'
model.vocabulary.min_count = 1   #修改之前訓練時的設定(新增的句子較少所以 詞的最小出現次數改為1)
with open(file_path,'r',encoding='utf-8') as f:
    sentence = []
    for line in f:
        sentence.append(line.strip('\n').strip(' ').split(' '))
    model.build_vocab(sentence, update=True)#训练该行
    model.train(sentence,total_examples= model.corpus_count,epochs= model.epochs)
print(len(model.wv.index2word))
model.wv.most_similar(positive='傻眼貓咪',topn=5)

692778


[('輻射區', 0.259126216173172),
 ('林耕仁', 0.2490512579679489),
 ('間', 0.24377873539924622),
 ('佔領區', 0.2437250018119812),
 ('34.83', 0.24121515452861786)]

In [ ]:
#文件訓練word2vec，儲存
def Word2vec_train(file_path,save_path,dir_path=None,save_name='word2vec_model',replace_old=False,
                   model_size=300,model_window=10,model_min_count=5,**kw):
    """
    batch train usage: set dir_path、save_name, file_path = None, save_path = None
    if Multiple files using dir_path
    """
    from gensim.models import word2vec
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    # https://radimrehurek.com/gensim/models/word2vec.html
    if file_path != None:
        #單檔案
        sentences = word2vec.LineSentence(file_path)
        model = word2vec.Word2Vec(sentences, size=model_size,window=model_window,min_count=model_min_count,**kw)
        #保存模型，供日後使用
        model.save(save_path)
    if dir_path != None and file_path == None:
        #多檔案
        sentences = word2vec.PathLineSentences(dir_path)
        model = word2vec.Word2Vec(sentences, size=model_size,window=model_window,min_count=model_min_count,**kw)
        #保存模型，供日後使用
        model.save(os.path.join(dir_path,save_name))
    #模型讀取方式
    # model = word2vec.Word2Vec.load("your_model_name") 

In [ ]:
import logging

from gensim.models import word2vec

def main(file_path,save_path):

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    # https://radimrehurek.com/gensim/models/word2vec.html
    sentences = word2vec.LineSentence(file_path)
    #多檔案
#     sentences = word2vec.PathLineSentences(segment_dir)
    model = word2vec.Word2Vec(sentences, size=280,window=10,min_count=5)

    #保存模型，供日後使用
    model.save(save_path)

    #模型讀取方式
    # model = word2vec.Word2Vec.load("your_model_name")

if __name__ == "__main__":
    file_path = r'D:\ml_data\data_set\NLP\wiki_seg.txt'
    save_path = r'D:\ml_data\data_set\NLP\wiki_word2vec3.model'
    main(file_path,save_path)

In [7]:
import torch
import pandas as pd
import os
from opencc import OpenCC
import re
import logging
import jieba
import numpy as np
from gensim.models import word2vec
save_path = r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\sentence\wiki_word2vec0.model'
word_vec_model = word2vec.Word2Vec.load(save_path)

In [8]:
dir_path = r'D:\ml_data\Data_set\NLP\chinese_sentence_collection\Tidied_Corpus\chat'
file_name = r'merge_chat.txt'
file_path = os.path.join(dir_path,file_name)

In [9]:
def trim(file_path,max_word_num):
    with open(file_path,'r',encoding='utf-8') as f:
        new_list = []
        for chat in f:
            #依行讀取
            save_chat = chat.strip('\n').split('\t')
            X = save_chat[0].split(' ')
            Y = save_chat[1].split(' ')
            #如果詞數大於30
            if len(X) < max_word_num and len(Y) < max_word_num:
                for num in range(max_word_num-len(X)):
                    X.append(0)
                for num in range(max_word_num-len(Y)):
                    Y.append(0)
                new_list.append([X,Y])
            if len(save_chat) < 2:
                print(save_chat)
                break
        return new_list
chat_data = trim(file_path,max_word_num=20)

In [15]:
# %%time
def word_to_vec(index):
    padding = np.zeros((1,300),dtype=np.float32)
    chat = chat_data[index]
    #X
    vec_array = np.zeros((1,300),dtype=np.float32)
    for word in chat[0]:
        if word == 0:
            vec = padding
            vec_array = np.concatenate((vec_array,vec),axis=0)
        else:
            try:
                vec = np.array(word_vec_model.wv.get_vector(word),np.float32).reshape(1,300)
                vec_array = np.concatenate((vec_array,vec),axis=0)
            except KeyError:
                vec = padding
                vec_array = np.concatenate((vec_array,vec),axis=0)
    X = vec_array[1:]
    #Y
    vec_array = np.zeros((1,300),dtype=np.float32)
    for word in chat[1]:
        if word == 0:
            vec = padding
            vec_array = np.concatenate((vec_array,vec),axis=0)
        else:
            try:
                vec = np.array(word_vec_model.wv.get_vector(word),np.float32).reshape(1,300)
                vec_array = np.concatenate((vec_array,vec),axis=0)
            except KeyError:
                vec = padding
                vec_array = np.concatenate((vec_array,vec),axis=0)
    Y = vec_array[1:]
    return X,Y

X,Y = word_to_vec(0)
len(chat_data)

94319

In [15]:
# %%time
padding = np.zeros((1,300),dtype=np.float32)
X = []
Y = []
for chat in chat_data[:1]:
    vec_array = np.zeros((1,300),dtype=np.float32)
    for word in chat[0]:
        if word == 0:
            vec = padding
            vec_array = np.concatenate((vec_array,vec),axis=0)
        else:
            try:
                vec = np.array(word_vec_model.wv.get_vector(word),np.float32).reshape(1,300)
                vec_array = np.concatenate((vec_array,vec),axis=0)
            except KeyError:
                vec = zero_vec
                vec_array = np.concatenate((vec_array,vec),axis=0)
    X.append(vec_array[1:])
    #-----#
    vec_array = np.zeros((1,300),dtype=np.float32)
    for word in chat[1]:
        if word == 0:
            vec = padding
            vec_array = np.concatenate((vec_array,vec),axis=0)
        else:
            try:
                vec = np.array(word_vec_model.wv.get_vector(word),np.float32).reshape(1,300)
                vec_array = np.concatenate((vec_array,vec),axis=0)
            except KeyError:
                vec = zero_vec
                vec_array = np.concatenate((vec_array,vec),axis=0)
    Y.append(vec_array[1:])
X = np.array(X)
Y = np.array(Y)

(1, 20, 300)